- 保监会 分类模型 2 训练 

# 基本设置

In [1]:
##load packages, needed
# encoding=utf-8

import jieba
from jieba import analyse
import sys
import re
import time
import string
from sklearn import feature_extraction
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif 
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

import pickle

%matplotlib inline
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
class StatsFeatures_cor(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.neg = set()
        f = open("corpus/neg_words_20180704.txt","r+", encoding='UTF-8')
        for content in f:
            self.neg.add(content.strip())
        f.close()
        
        self.company = set() # 公司
        f = open("corpus/bank_company_20180814.txt","r+", encoding='UTF-8')
        for content in f:
            self.company.add(content.strip())
        f.close()

        self.regulators = set() # 监管机构及领导
        f = open("corpus/bank_regulators_20180815.txt","r+", encoding='UTF-8')
        for content in f:
            self.regulators.add(content.strip())
        f.close()    
        
        #初始化字典liwc
        self.liwc = {} 
        f2 = open("corpus/scliwc.txt",'r', encoding = 'gb18030')
        for ii in f2:     #ii在scliwc.txt中循环
            i = ii.strip().split() 
            self.liwc[i[0]] = i[1:len(i)]
        f2.close      
        
        self.category = set()
        for i in list(self.liwc.values()):
            for j in i:
                self.category.add(j)         
        
    def fit(self, X, y=None):
        return self

    def getcnt(self,x):        
        return len(list(set(x)))

    def getnegcnt(self,x):
        negcnt = 0
        words = x.split()
        for w in words:
            if w in self.neg:
                negcnt = negcnt+1
        return negcnt
    
    def getorgcnttf(self,x):
        companycnt=0
        companytf=0
        regcnt = 0
        regtf = 0
        
        words = x.split()
        words_set=set(words)
        for w in words_set:
            if w in self.company:
                companycnt = companycnt+1
                companytf=companytf+words.count(w)
                
            if w in self.regulators:
                regcnt = regcnt+1
                regtf=regtf+words.count(w)            
                
        return companycnt, companytf, regcnt, regtf
    
    def transform(self, X):
        data = []
        for x in X:
            if len(x) == 0:
                length  = 1
            else :
                length = len(x)
                
            companycnt, companytf, regcnt, regtf=self.getorgcnttf(x)
            
#             words = x.split()
#             psy = []
#             for w in words:
#                 if w in self.liwc: #是否liwc字典包含分词结果列表words的哪些分词
#                     psy += self.liwc[w] 
            
#             cat_tf = []
#             for cat in self.category:
#                 cat_tf.append(psy.count(cat)) 
                
            data.append([len(x),self.getcnt(x),self.getcnt(x)/length,
                         self.getnegcnt(x),self.getnegcnt(x)/length,
                         companycnt, companytf, regcnt, regtf]) #  + cat_tf           
        return data

In [3]:
def getkeywords(X, N = 1000):
    '''
    训练时生成，合并所有记录，取N个关键词
    '''
    textrank = analyse.textrank

    text_combined = ' '.join(X)
    keywords = textrank(text_combined, topK = N)
    print('keywords num: ', len(keywords))
    if len(keywords) < N : 
        N  = len(keywords)

    if keywords:
        f = open("corpus/keywords.txt","w+", encoding='UTF-8')
        for content in keywords:
            content = content.strip()
            f.write(content + '\n')
        f.close()

In [4]:
class Statskeywords(BaseEstimator, TransformerMixin):
    
    def __init__(self, topk = 100):
        self.topk = topk
        
        self.keywords = set()
        f = open("corpus/keywords.txt","r+", encoding='UTF-8')
        num = 0
        for content in f:
            if num < topk:
                self.keywords.add(content.strip().replace('\n', ''))
            num += 1
        f.close() 
    
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X):
        '''
        文本中关键词的词频
        '''                        
        data = []
        for x in X:
            words = x.split()
            word_tf = []
            keycnt = 0
            for kw in self.keywords:
                word_tf.append(words.count(kw)) # 各个关键词的词频
                if kw in words:keycnt+=1
            word_tf.append(keycnt) # 关键词的个数             
                
            data.append(word_tf)       
            
        return data        

## 上一版模型读入

In [5]:
# 上一版模型
# from sklearn.externals import joblib
# pipeline_old = joblib.load( "model/circ_8classifier_0808.pkl.z") 

# 导入数据

In [6]:
title = []
filename = 'data/titles0.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    title.append(f.strip().replace('\n', ''))
fid.close()
print(len(title))

6526


In [7]:
title[:2]

['小白学 保险 基础 篇 买好 宝宝 保险 第一步 该 做', '过 诉讼时效 债务 如何 处理 最高法院 答复 全']

In [8]:
content = []
filename = 'data/contents0.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    content.append(f.strip().replace('\n', ''))
fid.close()
print(len(content))
# content[:2]

6526


In [9]:
title_content = [t + ' ' + c for t,c in zip(title, content)]
print(len(title_content))

6526


In [10]:
label_dic={'监管':1,'行业':2,'产品销售':3,'资本市场':4,'公司内部管理':5,'消费服务':6,'其他相关报道':7,'噪音':8}
class_name_dict = {v: k for k, v in label_dic.items()}
class_name_dict

{1: '监管',
 2: '行业',
 3: '产品销售',
 4: '资本市场',
 5: '公司内部管理',
 6: '消费服务',
 7: '其他相关报道',
 8: '噪音'}

In [11]:
label = []
filename = 'data/labels.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    label.append(f.strip().replace('\n', ''))
fid.close()
print(len(label))
label[:5]

6526


['噪音', '噪音', '噪音', '噪音', '噪音']

In [12]:
label = [label_dic[l] for l in label]
label[:5]

[8, 8, 8, 8, 8]

# 训练模型

## 分割训练集、测试集

In [13]:
X_train, X_test, y_train, y_test = train_test_split(title_content, label, test_size=0.3, random_state=42)
print('训练集：',len(y_train))
print('训练集-各类数量：',Counter(y_train))
print('测试集：',len(y_test))
print('测试集-各类数量：',Counter(y_test))

训练集： 4568
训练集-各类数量： Counter({1: 939, 2: 684, 6: 603, 8: 549, 7: 523, 4: 453, 5: 436, 3: 381})
测试集： 1958
测试集-各类数量： Counter({1: 395, 2: 300, 6: 274, 8: 251, 7: 197, 5: 196, 4: 189, 3: 156})


## 生成词典

In [13]:
if 0: 
    from jieba import analyse
    getkeywords(title_content, N = 1000)

### 分类生成词典

In [14]:
# j = 1
N = 500
class_key_dict = {}
key_dict = {}
for j in range(1,9):
    str_list = [m for m,n in zip(title_content, label) if n == j]
    print(len(str_list))
    print(label.count(j))
    textrank = analyse.textrank
    text_combined = ' '.join(str_list)
    keywords = textrank(text_combined, topK = N)
    print('keywords num: ', len(keywords))
#     print('%s -- %s: '%(j, class_name_dict[j]) , keywords)
    class_key_dict[j] = keywords
    for key in keywords:
        if key not in key_dict:            
            key_dict[key] = 1
        else :
            key_dict[key] += 1
    

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


1334
1334


Loading model cost 0.910 seconds.
Prefix dict has been built succesfully.


keywords num:  500
984
984
keywords num:  500
537
537
keywords num:  500
642
642
keywords num:  500
632
632
keywords num:  500
877
877
keywords num:  500
720
720
keywords num:  500
800
800
keywords num:  500


In [15]:
key_count_list = [k for k,v in key_dict.items() if v > 3]
print(len(key_count_list))
print(len(key_dict.values()))
print(list(key_dict.values()))

385
1567
[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 8, 6, 8, 5, 8, 5, 7, 7, 8, 4, 8, 8, 6, 6, 5, 7, 8, 4, 7, 8, 8, 7, 5, 5, 3, 1, 2, 6, 8, 7, 5, 4, 8, 7, 4, 5, 8, 1, 3, 5, 2, 8, 5, 8, 7, 8, 3, 5, 1, 2, 7, 6, 7, 3, 4, 4, 7, 7, 8, 6, 7, 8, 4, 7, 7, 7, 5, 6, 3, 5, 7, 8, 4, 5, 8, 3, 8, 6, 8, 8, 7, 5, 7, 8, 8, 3, 6, 8, 5, 5, 1, 8, 6, 2, 8, 5, 6, 6, 6, 6, 1, 5, 7, 5, 8, 5, 7, 7, 8, 4, 6, 5, 4, 3, 6, 7, 5, 3, 5, 8, 6, 2, 7, 6, 6, 7, 6, 3, 6, 7, 7, 4, 1, 8, 6, 6, 4, 2, 7, 1, 8, 5, 1, 5, 4, 7, 3, 1, 4, 6, 4, 2, 5, 4, 6, 3, 7, 4, 5, 3, 1, 7, 7, 7, 7, 2, 4, 8, 1, 6, 6, 4, 4, 2, 1, 3, 2, 4, 6, 3, 7, 1, 3, 8, 8, 4, 3, 7, 2, 6, 7, 8, 2, 7, 7, 5, 6, 5, 6, 5, 5, 1, 3, 6, 1, 4, 1, 6, 8, 5, 4, 2, 1, 7, 8, 7, 6, 3, 8, 8, 8, 7, 1, 1, 3, 8, 5, 3, 6, 4, 4, 6, 4, 2, 4, 7, 6, 6, 1, 6, 3, 4, 6, 8, 2, 8, 4, 1, 2, 8, 6, 4, 7, 6, 5, 5, 2, 3, 3, 3, 6, 3, 4, 4, 1, 5, 7, 4, 3, 5, 5, 3, 5, 3, 5, 4, 7, 6, 8, 1, 4, 7, 1, 2, 4, 5, 4, 2, 1, 6, 2, 1, 3, 1, 3, 4, 3, 3, 1, 3, 4, 4, 5, 1, 6, 6, 6, 2, 3, 5, 

In [16]:
with pd.ExcelWriter('data/word_class_freq.xlsx') as writer:
    for j in class_key_dict:
        class_name = class_name_dict[j]
        tmp_data = pd.DataFrame(class_key_dict[j])
        tmp_data.columns = ['word']
        tmp_data['class_freq'] = tmp_data['word'].apply(lambda x: key_dict[x])
        tmp_data.to_excel(writer, class_name)
    writer.save() 

In [17]:
keywords = []
for j in range(1,9):
    print('%s --'%j)
    keyword_list = class_key_dict[j]
    print('去除前：', len(keyword_list))
    for k in key_count_list:        
        if k in keyword_list:
            keyword_list.remove(k)
    print('去除后：', len(keyword_list)) 
    
    keywords += keyword_list

keywords = list(set(keywords))
print(keywords[:10])
print('len(keywords): ', len(keywords))
f = open("corpus/keywords.txt","w+", encoding='UTF-8')
for content in keywords:
    content = content.strip()
    f.write(content + '\n')
f.close()    

1 --
去除前： 500
去除后： 180
2 --
去除前： 500
去除后： 166
3 --
去除前： 500
去除后： 272
4 --
去除前： 500
去除后： 208
5 --
去除前： 500
去除后： 178
6 --
去除前： 500
去除后： 300
7 --
去除前： 500
去除后： 253
8 --
去除前： 500
去除后： 195
['登记', '技巧', '无需', '政治', '钱包', '假币', '讲座', '交叉', '主席', '趋势']
len(keywords):  1182


## LDA: 主题模型选特征

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    #打印每个主题下权重较高的term
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
    #打印主题-词语分布矩阵
    print(model.components_)

In [81]:
from sklearn.decomposition import LatentDirichletAllocation

tf_vectorizer  = CountVectorizer(max_df=0.95, min_df=2)
tf = tf_vectorizer .fit_transform(X_train)

In [35]:
# len(tf.data), tf.data

# tf_feature_names = tf_vectorizer.get_feature_names()
# print(tf_feature_names[:10])
# len(tf_feature_names)

# n_top_words=20
# tf_feature_names = tf_vectorizer.get_feature_names()
# print_top_words(lda, tf_feature_names, n_top_words)

In [84]:
lda = LatentDirichletAllocation(n_topics=20,
                                max_iter=50,
                                learning_method='batch',
                                random_state=0)
docres = lda.fit_transform(tf)
lda.perplexity(tf)

In [85]:
docres

array([[3.90625003e-04, 9.92578125e-01, 3.90625003e-04, ...,
        3.90625095e-04, 3.90625002e-04, 3.90625004e-04],
       [4.50450455e-04, 1.28539163e-01, 7.50649189e-01, ...,
        4.50450465e-04, 4.50450457e-04, 4.50450459e-04],
       [6.44618417e-02, 7.77604998e-05, 7.77604987e-05, ...,
        2.78697680e-02, 2.06295485e-02, 7.77604990e-05],
       ...,
       [1.21359227e-04, 1.21359227e-04, 1.21359225e-04, ...,
        1.21359226e-04, 1.21359226e-04, 1.21359225e-04],
       [4.32523919e-02, 5.10204087e-04, 5.10204084e-04, ...,
        3.71540782e-01, 5.10204085e-04, 5.10204082e-04],
       [9.32835840e-05, 9.32835834e-05, 9.32835835e-05, ...,
        9.32835830e-05, 1.79299844e-01, 2.00158643e-01]])

In [86]:
lda.components_

array([[5.00000001e-02, 5.56502649e+00, 5.00000000e-02, ...,
        5.00000000e-02, 5.00000000e-02, 5.00000006e-02],
       [1.01622379e+01, 5.00000005e-02, 5.00000000e-02, ...,
        5.00000006e-02, 5.00000003e-02, 5.00000002e-02],
       [5.00000003e-02, 5.00000002e-02, 5.00000000e-02, ...,
        5.00000000e-02, 5.00000000e-02, 5.00000040e-02],
       ...,
       [5.00000001e-02, 5.00000004e-02, 5.00000000e-02, ...,
        5.00000000e-02, 5.00000000e-02, 5.00000001e-02],
       [5.00000012e-02, 5.00000010e-02, 5.00000000e-02, ...,
        5.00000028e-02, 5.00000007e-02, 5.00000001e-02],
       [2.06829458e+02, 5.76482636e+01, 5.00000000e-02, ...,
        5.00000000e-02, 5.00000000e-02, 5.00000002e-02]])

In [88]:
doc_topic_dist = lda.transform(tf)

In [ ]:
from time import time
n_topics = [200,] #range(20, 75, 5)
perplexityLst = [1.0]*len(n_topics)

#训练LDA并打印训练时间
lda_models = []
for idx, n_topic in enumerate(n_topics):
    lda = LatentDirichletAllocation(n_topics=n_topic,
                                    max_iter=1000,
                                    learning_method='batch',
                                    evaluate_every=200,
#                                    perp_tol=0.1, #default                                       
#                                    doc_topic_prior=1/n_topic, #default
#                                    topic_word_prior=1/n_topic, #default
                                    verbose=0)
    t0 = time()
    lda.fit(tf)
    perplexityLst[idx] = lda.perplexity(tf)
    lda_models.append(lda)
    print("# of Topic: %d, " % n_topics[idx])
    print("done in %0.3fs, N_iter %d, " % ((time() - t0), lda.n_iter_))
    print("Perplexity Score %0.3f" % perplexityLst[idx])

In [ ]:
#打印最佳模型
best_index = perplexityLst.index(min(perplexityLst))
best_n_topic = n_topics[best_index]
best_model = lda_models[best_index]
print("Best # of Topic: ", best_n_topic)

## xgboost

### train

In [18]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=2000))
        ])),
        ('len_stats', StatsFeatures_cor()),
        ('tf', Pipeline([
            ('tf_k', Statskeywords(topk = 5000)),
            ('chi', SelectKBest(chi2, k=500))])),
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=8))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9997810858143608


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9...tate=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

In [19]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.9044943820224719
             precision    recall  f1-score   support

          1       0.92      0.96      0.94       395
          2       0.85      0.88      0.87       300
          3       0.95      0.88      0.91       156
          4       0.85      0.81      0.83       189
          5       0.87      0.79      0.83       196
          6       0.92      0.95      0.93       274
          7       0.93      0.93      0.93       197
          8       0.94      0.95      0.94       251

avg / total       0.90      0.90      0.90      1958

confusion_matrix: 
[[380  10   0   1   1   2   0   1]
 [ 13 265   2   9   9   1   0   1]
 [  2   6 137   2   3   2   2   2]
 [  4  10   0 153   3   9   1   9]
 [  7  10   0   9 154   8   7   1]
 [  3   5   2   0   2 259   3   0]
 [  1   1   3   1   4   1 184   2]
 [  1   3   0   6   1   1   0 239]]


### test

In [23]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.854006586169045
             precision    recall  f1-score   support

          1       0.63      0.73      0.68        52
          2       0.82      0.91      0.86       299
          3       0.82      0.61      0.70        38
          4       0.91      0.60      0.73        88
          5       0.76      0.62      0.68        63
          6       0.90      0.95      0.93        99
          7       0.93      0.83      0.87        46
          8       0.92      0.98      0.95       226

avg / total       0.86      0.85      0.85       911

confusion_matrix: 
[[ 38  13   0   0   0   0   0   1]
 [ 19 272   2   0   3   1   0   2]
 [  0   8  23   0   1   0   2   4]
 [  0  20   0  53   2   3   0  10]
 [  1  13   1   2  39   5   1   1]
 [  2   0   1   0   2  94   0   0]
 [  0   2   1   2   3   0  38   0]
 [  0   2   0   1   1   1   0 221]]


In [16]:
# 上一版模型 
y_pred_class = pipeline_old.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.9376872378669863
             precision    recall  f1-score   support

          1       0.96      0.95      0.96       425
          2       0.88      0.88      0.88       320
          3       0.89      0.93      0.91       215
          4       0.92      0.92      0.92       206
          5       0.95      0.83      0.89       322
          6       0.95      0.95      0.95       407
          7       0.94      0.98      0.96       432
          8       0.95      0.97      0.96      1011

avg / total       0.94      0.94      0.94      3338

confusion_matrix: 
[[404   7   0   0   2   0   1  11]
 [  3 281   8   5   4   6   4   9]
 [  0   6 199   0   0   3   3   4]
 [  2   4   2 189   4   0   0   5]
 [  5  16   4   3 267   7   8  12]
 [  0   1   6   0   1 387   6   6]
 [  1   0   1   0   0   1 425   4]
 [  6   4   4   8   3   5   3 978]]


### KFold

In [14]:
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=0)
title_content = np.array(title_content)
label = np.array(label)

In [15]:
num = 0
for train_index , test_index in kf.split(title_content):
    print('---- %s'%(num+1))
    X_train,X_test = title_content[train_index], title_content[test_index]
    y_train,y_test = label[train_index], label[test_index]
    
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('tf_idf', Pipeline([
                ('counts', CountVectorizer(max_df=0.95, min_df=2)),
                ('tf_idf', TfidfTransformer()),
                ('chi', SelectKBest(chi2, k=20000))
            ])),
            ('tf', Statskeywords(topk = 1000)),
            ('len_stats', StatsFeatures_cor())
        ])),
        ('standard', StandardScaler(with_mean=False)),
        ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=8))
    ])
    pipeline.fit(X_train, y_train)
    print(pipeline.score(X_train, y_train))    
    
    y_pred_class = pipeline.predict(X_test)
    print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
    print(metrics.classification_report(y_test, y_pred_class))
    print('confusion_matrix: ')
    print( metrics.confusion_matrix(y_test, y_pred_class))    
    
    num += 1

---- 1
0.9991673605328892
accuracy_score:  0.8408788282290279
             precision    recall  f1-score   support

          1       0.83      0.88      0.86       286
          2       0.52      0.41      0.46        58
          3       0.76      0.50      0.60        58
          4       0.88      0.78      0.83        98
          5       0.82      0.75      0.78       126
          6       0.71      0.80      0.75        59
          7       0.85      0.81      0.83       151
          8       0.88      0.93      0.90       666

avg / total       0.84      0.84      0.84      1502

confusion_matrix: 
[[253   5   2   1   1   0   2  22]
 [ 11  24   0   0   2   3   1  17]
 [  6   2  29   1   4   2   5   9]
 [  2   3   0  76   6   0   1  10]
 [  7   3   1   1  94   3   8   9]
 [  0   1   1   0   1  47   1   8]
 [  3   3   1   0   3   8 122  11]
 [ 21   5   4   7   4   3   4 618]]
---- 2
0.9993338884263114
accuracy_score:  0.8468708388814914
             precision    recall  f1-score 

# 调参

In [26]:
score_1_best_para = {}
score_2_best_para = {}
cv = 3
score_1 = 'accuracy'
score_2 = 'recall_macro'

In [27]:
combined_features = FeatureUnion([
                                    ('tf_idf', Pipeline([
                                        ('counts', CountVectorizer()),
                                        ('tfidf', TfidfTransformer()),
                                        ('chi', SelectKBest(chi2, k=20000))
                                        ])),
                                    ('tf', Pipeline([
                                        ('tf_k', Statskeywords(topk = 5000)),
                                        ('chi', SelectKBest(chi2, k=500))])),
                                    ('len_stats', StatsFeatures_cor())
                                ])

## topk、chi_k

In [28]:
pipeline = Pipeline([('features', combined_features), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0,
                                                  objective='multi:softmax', num_class=8))])

param_grid_0 = dict(features__tf_idf__chi__k = [2000, 5000, 20000, 40000 ],
                   features__tf__chi__k = [50,100, 500]) 
param_grid_0

{'features__tf_idf__chi__k': [2000, 5000, 20000, 40000],
 'features__tf__chi__k': [50, 100, 500]}

In [29]:
clf = GridSearchCV(pipeline, param_grid=param_grid_0, 
                   cv = cv, scoring=score_1, n_jobs=-1, verbose=10)
clf.fit(X_train, y_train)
score_1_best_para['tf_idf__chi__k'] = clf.best_params_['features__tf_idf__chi__k']
score_1_best_para['tf__chi__k'] = clf.best_params_['features__tf__chi__k']
print('score_1_tf_idf__chi__k: ', score_1_best_para['tf_idf__chi__k'])
print('score_1_tf__chi__k: ', score_1_best_para['tf__chi__k'])
clf.grid_scores_, clf.best_params_, clf.best_score_

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] features__tf__chi__k=50, features__tf_idf__chi__k=2000 ..........
[CV] features__tf__chi__k=50, features__tf_idf__chi__k=2000 ..........
[CV] features__tf__chi__k=50, features__tf_idf__chi__k=2000 ..........
[CV] features__tf__chi__k=50, features__tf_idf__chi__k=5000 ..........
[CV]  features__tf__chi__k=50, features__tf_idf__chi__k=2000, score=0.8618699499807618, total= 8.4min
[CV] features__tf__chi__k=50, features__tf_idf__chi__k=5000 ..........
[CV]  features__tf__chi__k=50, features__tf_idf__chi__k=2000, score=0.8593990755007704, total= 8.4min
[CV]  features__tf__chi__k=50, features__tf_idf__chi__k=2000, score=0.8700347088314694, total= 8.4min
[CV] features__tf__chi__k=50, features__tf_idf__chi__k=5000 ..........
[CV] features__tf__chi__k=50, features__tf_idf__chi__k=20000 .........
[CV]  features__tf__chi__k=50, features__tf_idf__chi__k=5000, score=0.8680261639091958, total= 9.7min
[CV] features__tf__chi__k=50, feat

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 26.2min


[CV]  features__tf__chi__k=50, features__tf_idf__chi__k=5000, score=0.8597842835130971, total= 9.9min
[CV] features__tf__chi__k=50, features__tf_idf__chi__k=40000 .........
[CV]  features__tf__chi__k=50, features__tf_idf__chi__k=20000, score=0.8672566371681416, total=11.9min
[CV] features__tf__chi__k=50, features__tf_idf__chi__k=40000 .........
[CV]  features__tf__chi__k=50, features__tf_idf__chi__k=20000, score=0.8617103235747303, total=12.0min
[CV] features__tf__chi__k=50, features__tf_idf__chi__k=40000 .........
[CV]  features__tf__chi__k=50, features__tf_idf__chi__k=20000, score=0.8708060161974547, total=12.4min
[CV] features__tf__chi__k=100, features__tf_idf__chi__k=2000 .........
[CV]  features__tf__chi__k=50, features__tf_idf__chi__k=40000, score=0.8641785302039245, total=14.1min


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 45.2min


[CV] features__tf__chi__k=100, features__tf_idf__chi__k=2000 .........
[CV]  features__tf__chi__k=50, features__tf_idf__chi__k=40000, score=0.8601694915254238, total=14.8min
[CV] features__tf__chi__k=100, features__tf_idf__chi__k=2000 .........
[CV]  features__tf__chi__k=50, features__tf_idf__chi__k=40000, score=0.8731199382954107, total=15.1min
[CV] features__tf__chi__k=100, features__tf_idf__chi__k=5000 .........
[CV]  features__tf__chi__k=100, features__tf_idf__chi__k=2000, score=0.8626394767218161, total=12.1min
[CV] features__tf__chi__k=100, features__tf_idf__chi__k=5000 .........
[CV]  features__tf__chi__k=100, features__tf_idf__chi__k=2000, score=0.8578582434514638, total=11.8min
[CV] features__tf__chi__k=100, features__tf_idf__chi__k=5000 .........
[CV]  features__tf__chi__k=100, features__tf_idf__chi__k=2000, score=0.8650212109525646, total=11.9min
[CV] features__tf__chi__k=100, features__tf_idf__chi__k=20000 ........
[CV]  features__tf__chi__k=100, features__tf_idf__chi__k=50

...

[Parallel(n_jobs=-1)]: Done  33 out of  36 | elapsed: 165.5min remaining: 15.0min


[CV]  features__tf__chi__k=500, features__tf_idf__chi__k=40000, score=0.862254713351289, total=23.1min
[CV]  features__tf__chi__k=500, features__tf_idf__chi__k=40000, score=0.8601694915254238, total=19.3min
[CV]  features__tf__chi__k=500, features__tf_idf__chi__k=40000, score=0.870420362514462, total=16.0min


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 180.9min finished


score_1_tf_idf__chi__k:  20000
score_1_tf__chi__k:  50


([mean: 0.86376, std: 0.00454, params: {'features__tf__chi__k': 50, 'features__tf_idf__chi__k': 2000},
  mean: 0.86505, std: 0.00373, params: {'features__tf__chi__k': 50, 'features__tf_idf__chi__k': 5000},
  mean: 0.86659, std: 0.00374, params: {'features__tf__chi__k': 50, 'features__tf_idf__chi__k': 20000},
  mean: 0.86582, std: 0.00541, params: {'features__tf__chi__k': 50, 'features__tf_idf__chi__k': 40000},
  mean: 0.86184, std: 0.00298, params: {'features__tf__chi__k': 100, 'features__tf_idf__chi__k': 2000},
  mean: 0.86454, std: 0.00274, params: {'features__tf__chi__k': 100, 'features__tf_idf__chi__k': 5000},
  mean: 0.86531, std: 0.00360, params: {'features__tf__chi__k': 100, 'features__tf_idf__chi__k': 20000},
  mean: 0.86569, std: 0.00215, params: {'features__tf__chi__k': 100, 'features__tf_idf__chi__k': 40000},
  mean: 0.86222, std: 0.00262, params: {'features__tf__chi__k': 500, 'features__tf_idf__chi__k': 2000},
  mean: 0.86466, std: 0.00384, params: {'features__tf__chi__k': 

In [56]:
pipeline = Pipeline([('features', combined_features), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0,
                                                  objective='multi:softmax', num_class=8))])

param_grid_0 = dict(features__tf_idf__chi__k = [2000, 5000, 20000, 40000 ],
                   features__tf__chi__k = [50,100, 500]) 
param_grid_0

{'features__tf_idf__chi__k': [2000, 5000, 20000, 40000],
 'features__tf__topk': [50, 100, 500]}

In [57]:
clf = GridSearchCV(pipeline, param_grid=param_grid_0, 
                   cv = cv, scoring=score_2, n_jobs=-1, verbose=10)
clf.fit(X_train, y_train)
score_2_best_para['tf_idf__chi__k'] = clf.best_params_['features__tf_idf__chi__k']
score_2_best_para['tf__chi__k'] = clf.best_params_['features__tf__chi__k']
print('score_2_tf_idf__chi__k: ', score_2_best_para['tf_idf__chi__k'])
print('score_2_tf__chi__k: ', score_2_best_para['tf__chi__k'])
clf.grid_scores_, clf.best_params_, clf.best_score_

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] features__tf__topk=50, features__tf_idf__chi__k=2000 ............
[CV] features__tf__topk=50, features__tf_idf__chi__k=2000 ............
[CV] features__tf__topk=50, features__tf_idf__chi__k=2000 ............
[CV] features__tf__topk=50, features__tf_idf__chi__k=5000 ............
[CV]  features__tf__topk=50, features__tf_idf__chi__k=2000, score=0.8231063761425004, total= 5.6min
[CV] features__tf__topk=50, features__tf_idf__chi__k=5000 ............
[CV]  features__tf__topk=50, features__tf_idf__chi__k=2000, score=0.8118461781032549, total= 5.8min
[CV] features__tf__topk=50, features__tf_idf__chi__k=5000 ............
[CV]  features__tf__topk=50, features__tf_idf__chi__k=2000, score=0.8300723296263293, total= 5.8min
[CV] features__tf__topk=50, features__tf_idf__chi__k=20000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=5000, score=0.8212898267295912, total= 8.1min
[CV] features__tf__topk=50, features__tf_i

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 14.2min


[CV]  features__tf__topk=50, features__tf_idf__chi__k=5000, score=0.810622908581124, total= 6.9min
[CV] features__tf__topk=50, features__tf_idf__chi__k=40000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=20000, score=0.8181799751454917, total=10.8min
[CV] features__tf__topk=50, features__tf_idf__chi__k=40000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=20000, score=0.82764145306582, total=11.2min
[CV] features__tf__topk=50, features__tf_idf__chi__k=40000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=20000, score=0.8150179879762034, total=11.6min
[CV] features__tf__topk=100, features__tf_idf__chi__k=2000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=40000, score=0.8186643180207618, total=13.5min


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 29.5min


[CV] features__tf__topk=100, features__tf_idf__chi__k=2000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=40000, score=0.8324023763764725, total=11.9min
[CV] features__tf__topk=100, features__tf_idf__chi__k=2000 ...........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=2000, score=0.8231063761425004, total= 5.8min
[CV] features__tf__topk=100, features__tf_idf__chi__k=5000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=40000, score=0.8149401664117953, total=13.0min
[CV] features__tf__topk=100, features__tf_idf__chi__k=5000 ...........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=2000, score=0.8300723296263293, total= 5.6min
[CV] features__tf__topk=100, features__tf_idf__chi__k=5000 ...........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=2000, score=0.8118461781032549, total= 5.5min
[CV] features__tf__topk=100, features__tf_idf__chi__k=20000 ..........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=5000, score=0.

...

[Parallel(n_jobs=-1)]: Done  33 out of  36 | elapsed: 87.2min remaining:  7.9min


[CV]  features__tf__topk=500, features__tf_idf__chi__k=40000, score=0.8186643180207618, total=12.7min
[CV]  features__tf__topk=500, features__tf_idf__chi__k=40000, score=0.8324023763764725, total=11.1min
[CV]  features__tf__topk=500, features__tf_idf__chi__k=40000, score=0.8149401664117953, total= 8.3min


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 95.7min finished


score_2_chi__k:  40000
score_2_topk:  50


([mean: 0.82168, std: 0.00751, params: {'features__tf__topk': 50, 'features__tf_idf__chi__k': 2000},
  mean: 0.82036, std: 0.00760, params: {'features__tf__topk': 50, 'features__tf_idf__chi__k': 5000},
  mean: 0.82028, std: 0.00536, params: {'features__tf__topk': 50, 'features__tf_idf__chi__k': 20000},
  mean: 0.82200, std: 0.00751, params: {'features__tf__topk': 50, 'features__tf_idf__chi__k': 40000},
  mean: 0.82168, std: 0.00751, params: {'features__tf__topk': 100, 'features__tf_idf__chi__k': 2000},
  mean: 0.82036, std: 0.00760, params: {'features__tf__topk': 100, 'features__tf_idf__chi__k': 5000},
  mean: 0.82028, std: 0.00536, params: {'features__tf__topk': 100, 'features__tf_idf__chi__k': 20000},
  mean: 0.82200, std: 0.00751, params: {'features__tf__topk': 100, 'features__tf_idf__chi__k': 40000},
  mean: 0.82168, std: 0.00751, params: {'features__tf__topk': 500, 'features__tf_idf__chi__k': 2000},
  mean: 0.82036, std: 0.00760, params: {'features__tf__topk': 500, 'features__tf_i

## combined_features

In [58]:
combined_features_1 = FeatureUnion([
                                    ('tf_idf', Pipeline([
                                        ('counts', CountVectorizer()),
                                        ('tfidf', TfidfTransformer()),
                                        ('chi', SelectKBest(chi2, k=score_1_best_para['chi__k']))
                                        ])),
                                    ('tf', Pipeline([
                                        ('tf_k', Statskeywords(topk = 5000)),
                                        ('chi', SelectKBest(chi2, k=score_1_best_para['topk']))])), 
                                    ('len_stats', StatsFeatures_cor())
                                ])

In [59]:
combined_features_2 = FeatureUnion([
                                    ('tf_idf', Pipeline([
                                        ('counts', CountVectorizer()),
                                        ('tfidf', TfidfTransformer()),
                                        ('chi', SelectKBest(chi2, k=score_2_best_para['chi__k']))
                                        ])),
                                    ('tf', Pipeline([
                                        ('tf_k', Statskeywords(topk = 5000)),
                                        ('chi', SelectKBest(chi2, k=score_2_best_para['topk']))])), 
                                    ('len_stats', StatsFeatures_cor())
                                ])

In [60]:
pipeline = Pipeline([('features', combined_features_1), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0, 
                                                  objective='multi:softmax', num_class=8))])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

0.998931195724783
accuracy_score:  0.878428927680798
             precision    recall  f1-score   support

          1       0.83      0.90      0.87       377
          2       0.79      0.75      0.77       283
          3       0.82      0.74      0.78       145
          4       0.86      0.83      0.84       201
          5       0.84      0.82      0.83       318
          6       0.92      0.93      0.92       434
          7       0.90      0.92      0.91       434
          8       0.92      0.91      0.92      1016

avg / total       0.88      0.88      0.88      3208

confusion_matrix: 
[[340  17   0   1   5   0   2  12]
 [ 14 213   5   6  10  11   5  19]
 [  1   6 108   1   5   7   5  12]
 [  5   5   0 167   9   0   3  12]
 [  5  16   4   3 262   5  12  11]
 [  1   4   7   1   3 402  10   6]
 [  4   1   2   1  11   6 399  10]
 [ 39   7   5  15   8   7   8 927]]


In [61]:
pipeline = Pipeline([('features', combined_features_2), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0, 
                                                  objective='multi:softmax', num_class=8))])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

0.9995991983967936
accuracy_score:  0.8806109725685786
             precision    recall  f1-score   support

          1       0.84      0.90      0.87       377
          2       0.81      0.76      0.79       283
          3       0.81      0.72      0.77       145
          4       0.85      0.84      0.84       201
          5       0.83      0.82      0.82       318
          6       0.92      0.93      0.93       434
          7       0.90      0.93      0.91       434
          8       0.92      0.91      0.92      1016

avg / total       0.88      0.88      0.88      3208

confusion_matrix: 
[[340  15   1   1   4   0   2  14]
 [ 14 215   5   5  12   9   6  17]
 [  2   6 105   0   5   6   9  12]
 [  5   4   0 168  10   0   3  11]
 [  6  18   4   4 261   5   9  11]
 [  1   1   9   1   2 405   8   7]
 [  4   0   1   1  10   5 402  11]
 [ 34   5   4  18  11   8   7 929]]


## n_estimators、learning_rate

In [62]:
pipeline = Pipeline([('features', combined_features_1), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0, 
                                                  objective='multi:softmax', num_class=8))])

param_grid_1 = dict(classifier__n_estimators = range(100,271,30), # 迭代次数/分类器个数
                   classifier__learning_rate=[0.01, 0.1, 0.3]) 
param_grid_1

{'classifier__n_estimators': range(10, 171, 30),
 'classifier__learning_rate': [0.01, 0.1, 0.3]}

In [63]:
clf = GridSearchCV(pipeline, param_grid=param_grid_1, 
                   cv = cv, scoring=score_1, n_jobs=-1)
clf.fit(X_train, y_train)
score_1_best_para['n_estimators'] = clf.best_params_['classifier__n_estimators']
score_1_best_para['learning_rate'] = clf.best_params_['classifier__learning_rate']
print('score_1_n_estimators: ', score_1_best_para['n_estimators'])
print('score_1_learning_rate: ', score_1_best_para['learning_rate'])
clf.grid_scores_, clf.best_params_, clf.best_score_

score_1_n_estimators:  160
score_1_learning_rate:  0.1


([mean: 0.79399, std: 0.00823, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 10},
  mean: 0.81296, std: 0.00664, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 40},
  mean: 0.82311, std: 0.00497, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 70},
  mean: 0.82685, std: 0.00471, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 100},
  mean: 0.83260, std: 0.00296, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 130},
  mean: 0.83447, std: 0.00176, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 160},
  mean: 0.82619, std: 0.00693, params: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 10},
  mean: 0.84756, std: 0.00574, params: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 40},
  mean: 0.85464, std: 0.00313, params: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 70},
  mean: 0.85919, std: 0.0037

In [64]:
pipeline = Pipeline([('features', combined_features_1), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0, 
                                                  n_estimators = score_1_best_para['n_estimators'],
                                                  learning_rate = score_1_best_para['learning_rate'],                                                  
                                                  objective='multi:softmax', num_class=8))])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

1.0
accuracy_score:  0.8812344139650873
             precision    recall  f1-score   support

          1       0.84      0.90      0.87       377
          2       0.80      0.77      0.79       283
          3       0.81      0.74      0.78       145
          4       0.86      0.84      0.85       201
          5       0.85      0.82      0.84       318
          6       0.92      0.93      0.92       434
          7       0.90      0.93      0.91       434
          8       0.92      0.91      0.92      1016

avg / total       0.88      0.88      0.88      3208

confusion_matrix: 
[[338  17   1   1   4   0   2  14]
 [ 13 219   5   5  10   9   5  17]
 [  2   7 108   0   4   7   5  12]
 [  3   5   0 168   9   0   3  13]
 [  6  15   5   3 261   5  11  12]
 [  1   4   7   1   3 402   9   7]
 [  4   0   2   1   9   6 402  10]
 [ 35   6   5  17   7   8   9 929]]


In [65]:
pipeline = Pipeline([('features', combined_features_2), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0, 
                                                  objective='multi:softmax', num_class=2))])

param_grid_1 = dict(classifier__n_estimators = range(30,161,20), # 迭代次数/分类器个数
                   classifier__learning_rate=[0.01, 0.1, 0.3]) 
param_grid_1

{'classifier__n_estimators': range(30, 161, 20),
 'classifier__learning_rate': [0.01, 0.1, 0.3]}

In [66]:
clf = GridSearchCV(pipeline, param_grid=param_grid_1, 
                   cv = cv, scoring=score_2, n_jobs=-1)
clf.fit(X_train, y_train)
score_2_best_para['n_estimators'] = clf.best_params_['classifier__n_estimators']
score_2_best_para['learning_rate'] = clf.best_params_['classifier__learning_rate']
print('score_2_n_estimators: ', score_2_best_para['n_estimators'])
print('score_2_learning_rate: ', score_2_best_para['learning_rate'])
clf.grid_scores_, clf.best_params_, clf.best_score_

score_2_n_estimators:  150
score_2_learning_rate:  0.1


([mean: 0.76202, std: 0.01255, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 30},
  mean: 0.76853, std: 0.00973, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 50},
  mean: 0.77944, std: 0.00715, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 70},
  mean: 0.77983, std: 0.00743, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 90},
  mean: 0.78303, std: 0.00422, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 110},
  mean: 0.78614, std: 0.00210, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 130},
  mean: 0.78836, std: 0.00123, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 150},
  mean: 0.79746, std: 0.00808, params: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 30},
  mean: 0.80711, std: 0.00718, params: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 50},
  mean: 0.81476, std: 0.007

In [67]:
pipeline = Pipeline([('features', combined_features_1), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0, 
                                                  n_estimators = score_2_best_para['n_estimators'],
                                                  learning_rate = score_2_best_para['learning_rate'],                                                  
                                                  objective='multi:softmax', num_class=8))])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

1.0
accuracy_score:  0.8793640897755611
             precision    recall  f1-score   support

          1       0.83      0.90      0.87       377
          2       0.80      0.76      0.78       283
          3       0.81      0.74      0.78       145
          4       0.85      0.83      0.84       201
          5       0.85      0.82      0.83       318
          6       0.92      0.93      0.92       434
          7       0.90      0.93      0.91       434
          8       0.92      0.91      0.91      1016

avg / total       0.88      0.88      0.88      3208

confusion_matrix: 
[[339  16   1   1   4   0   2  14]
 [ 14 216   5   6  10   9   5  18]
 [  2   7 108   1   4   7   5  11]
 [  4   5   0 167   9   0   3  13]
 [  6  15   5   3 260   5  12  12]
 [  1   4   7   1   3 402   9   7]
 [  4   0   2   1   9   6 402  10]
 [ 36   7   5  16   8   8   9 927]]


## max_depth、min_child_weight

In [68]:
pipeline = Pipeline([('features', combined_features_1), 
                     ('classifier', XGBClassifier(n_estimators = score_1_best_para['n_estimators'],
                                                  learning_rate = score_1_best_para['learning_rate'],
                                                  objective='multi:softmax', num_class=2))])

param_grid_2 = dict(classifier__max_depth=range(3,14,2), # 决策树最大深度
                    classifier__min_child_weight=[4, 5, 6]) 
param_grid_2

{'classifier__max_depth': range(3, 14, 2),
 'classifier__min_child_weight': [4, 5, 6]}

In [69]:
clf = GridSearchCV(pipeline, param_grid=param_grid_2, 
                   cv = cv, scoring=score_1, n_jobs=-1)
clf.fit(X_train, y_train)
score_1_best_para['max_depth'] = clf.best_params_['classifier__max_depth']
score_1_best_para['min_child_weight'] = clf.best_params_['classifier__min_child_weight']
print('score_1_max_depth: ', score_1_best_para['max_depth'])
print('score_1_min_child_weight: ', score_1_best_para['min_child_weight'])
clf.grid_scores_, clf.best_params_, clf.best_score_

score_1_max_depth:  5
score_1_min_child_weight:  4


([mean: 0.85691, std: 0.00251, params: {'classifier__max_depth': 3, 'classifier__min_child_weight': 4},
  mean: 0.85731, std: 0.00522, params: {'classifier__max_depth': 3, 'classifier__min_child_weight': 5},
  mean: 0.85424, std: 0.00512, params: {'classifier__max_depth': 3, 'classifier__min_child_weight': 6},
  mean: 0.86399, std: 0.00251, params: {'classifier__max_depth': 5, 'classifier__min_child_weight': 4},
  mean: 0.86012, std: 0.00566, params: {'classifier__max_depth': 5, 'classifier__min_child_weight': 5},
  mean: 0.86025, std: 0.00248, params: {'classifier__max_depth': 5, 'classifier__min_child_weight': 6},
  mean: 0.86039, std: 0.00154, params: {'classifier__max_depth': 7, 'classifier__min_child_weight': 4},
  mean: 0.86039, std: 0.00265, params: {'classifier__max_depth': 7, 'classifier__min_child_weight': 5},
  mean: 0.85892, std: 0.00384, params: {'classifier__max_depth': 7, 'classifier__min_child_weight': 6},
  mean: 0.85905, std: 0.00463, params: {'classifier__max_depth':

In [76]:
pipeline = Pipeline([('features', combined_features_1), 
                     ('classifier', XGBClassifier(gamma = 0, 
                                                  n_estimators = score_1_best_para['n_estimators'],
                                                  learning_rate = score_1_best_para['learning_rate'],        
                                                  max_depth = score_1_best_para['max_depth'],
                                                  min_child_weight = score_1_best_para['min_child_weight'],                                                   
                                                  objective='multi:softmax', num_class=8))])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

0.9931863727454909
accuracy_score:  0.8884039900249376
             precision    recall  f1-score   support

          1       0.85      0.89      0.87       377
          2       0.83      0.77      0.80       283
          3       0.83      0.76      0.79       145
          4       0.84      0.84      0.84       201
          5       0.86      0.84      0.85       318
          6       0.93      0.94      0.93       434
          7       0.90      0.93      0.92       434
          8       0.92      0.92      0.92      1016

avg / total       0.89      0.89      0.89      3208

confusion_matrix: 
[[337  15   1   5   4   0   2  13]
 [ 12 219   4   5  10  10   5  18]
 [  2   5 110   0   3   6   8  11]
 [  4   3   0 168   7   0   5  14]
 [  6  13   3   3 266   5  10  12]
 [  1   2   6   1   3 408   7   6]
 [  4   0   2   1   8   5 405   9]
 [ 30   6   6  18   7   5   7 937]]


In [72]:
score_1_best_para

{'chi__k': 2000,
 'topk': 50,
 'n_estimators': 160,
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_child_weight': 4}

In [30]:
combined_features_1 = FeatureUnion([
                                    ('tf_idf', Pipeline([
                                        ('counts', CountVectorizer()),
                                        ('tfidf', TfidfTransformer()),
                                        ('chi', SelectKBest(chi2, k=20000))
                                        ])),
                                    ('tf', Pipeline([
                                        ('tf_k', Statskeywords(topk = 5000)),
                                        ('chi', SelectKBest(chi2, k=50))])), 
                                    ('len_stats', StatsFeatures_cor())
                                ])

In [31]:
pipeline = Pipeline([('features', combined_features_1), 
                     ('classifier', XGBClassifier(gamma = 0, 
                                                  n_estimators = 160,
                                                  learning_rate = 0.1,        
                                                  max_depth = 5,
                                                  min_child_weight = 4,                                                   
                                                  objective='multi:softmax', num_class=8))])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

0.9952491011813046
accuracy_score:  0.8855602156980228
             precision    recall  f1-score   support

          1       0.87      0.87      0.87       425
          2       0.81      0.81      0.81       320
          3       0.88      0.79      0.83       215
          4       0.89      0.79      0.83       206
          5       0.83      0.81      0.82       322
          6       0.92      0.96      0.94       407
          7       0.93      0.92      0.92       432
          8       0.90      0.94      0.92      1011

avg / total       0.89      0.89      0.88      3338

confusion_matrix: 
[[370  13   0   4   3   0   4  31]
 [ 10 258   8   6  10   9   3  16]
 [  1  10 170   0   8   8   4  14]
 [  6   7   1 162  14   0   0  16]
 [  8  21   3   2 262   6  10  10]
 [  0   1   3   0   1 391   4   7]
 [  4   2   3   0  10   4 396  13]
 [ 26   6   6   9   7   6   4 947]]


# 模型保存

In [20]:
from sklearn.externals import joblib
joblib.dump(pipeline, "model/cbrc_8classifier_0827.pkl.z")

['model/cbrc_8classifier_0827.pkl.z']

# 保存本文件

In [56]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_2_train.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)